In [1]:
import re
import sys
import json


import sys
sys.dont_write_bytecode = True
sys.path.append('../')
from datatools.analyzer import *

from datatools.maneger import DataManager
from datatools.preproc import Preprocessor

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.26.5) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
from response.feature import Feature

In [3]:
F_path = "../X_y_data/response2/"
F_name = "Classify_F.pickle"
featureM = DataManager(F_path)
F = featureM.load_data(F_name)

success load : ../X_y_data/response2/Classify_F.pickle


In [4]:
model_path = "../models/response2/"
model_name = "Classify_M.pickle"
modelM = DataManager(model_path)
lr = modelM.load_data(model_name)

success load : ../models/response2/Classify_M.pickle


In [5]:
path = "../../corpus/func_labeled/"
datalist = ['DCM', 'DIT', 'IRS']
convs = read_conv(path, datalist)

In [36]:
usr_utt = []
y = []
for conv in convs:
    for i, ut in enumerate( conv ) :
        # if ut.is_system() and ut.is_exist_error():
        #     if conv[i-1].is_type_inclued("質問"):
        
        if not ut.is_system() :
            usr_utt.append(ut.utt)
            if ut.is_exist_type():
                y.append(1)
            else:
                y.append(0)


In [37]:
print("len of y:{0},  counts:{1}".format(len(y), y.count(1)))

len of y:2100,  counts:674


In [38]:
# 正解率を見ておく
from tqdm import tqdm
print(F.feature_num)
usr = []
for i, x_t_str in enumerate( tqdm(usr_utt) ):
    x = F.featurization(x_t_str)
    usr.append(x)
usr_ = np.array(usr)

  0%|          | 0/2100 [00:00<?, ?it/s]

20972


  2%|▏         | 34/2100 [00:06<06:57,  4.95it/s]

In [ ]:
y_pred = lr.predict(usr_)

In [ ]:
fixed_y = []
# {'YN': 0, 'WH': 1, 'please': 2, 'proposal': 3, 'plain': 4}
for p in y_pred:
    if p <= 3:
        fixed_y.append(1)
    else:
        fixed_y.append(0)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
print('confusion matrix = \n', confusion_matrix(y_true=y, y_pred=fixed_y))
print('accuracy = ', accuracy_score(y_true=y, y_pred=fixed_y))
print('precision = ', precision_score(y_true=y, y_pred=fixed_y))
print('recall = ', recall_score(y_true=y, y_pred=fixed_y))
print('f1 score = ', f1_score(y_true=y, y_pred=fixed_y))

confusion matrix = 
 [[2836  900]
 [ 103  571]]
accuracy =  0.772562358276644
precision =  0.3881713120326309
recall =  0.8471810089020771
f1 score =  0.5324009324009324


In [ ]:
# ここから Sentence-BERT vs 後ろ向き

from sentence_transformers import SentenceTransformer
from sentence_transformers import models ,losses
from sentence_transformers.readers import InputExample
from sentence_transformers.losses import TripletDistanceMetric, SoftmaxLoss
from sentence_transformers.evaluation import TripletEvaluator
from sentence_transformers.readers import TripletReader
from sentence_transformers.datasets import SentencesDataset
from torch.utils.data import DataLoader

In [ ]:
type_num = 0
neutral_num = 0
# 0: not error, 1: newtral, 2: error
X = []
for conv in convs:
    for i, ut in enumerate( conv ) :
        # if ut.is_system() and ut.is_exist_error():
        #     if conv[i-1].is_type_inclued("質問"):
        usr_utt.append(ut.utt)
        # ユーザ発話駆動
        if not ut.is_system():
            if ut.is_exist_type():
                type_num += 1
                # システムがエラー
                if conv[i+1].is_exist_error():
                    X.append( InputExample(texts=[ut.utt, conv[i+1].utt], label=2 ) )
                # エラーではない
                else:
                    X.append( InputExample(texts=[ut.utt, conv[i+1].utt], label=0 ) )
            # ニュートラル
            else :
                if neutral_num <= type_num:
                    if not conv[i+1].is_exist_error():
                        neutral_num += 1
                        X.append( InputExample(texts=[ut.utt, conv[i+1].utt], label=1 ) )


In [ ]:
X_train, X_test = train_test_split(X, train_size=0.7, random_state=4)

In [ ]:
from sentence_transformers import SentenceTransformer
from sentence_transformers import models

download_path = "../../corpus/"
# download_path = ""
transformer = models.Transformer(download_path+'cl-tohoku/bert-base-japanese-whole-word-masking')
pooling = models.Pooling(transformer.get_word_embedding_dimension(),    
  pooling_mode_mean_tokens=True,
  pooling_mode_cls_token=False, 
  pooling_mode_max_tokens=False
)
model = SentenceTransformer(modules=[transformer, pooling])

sentences = ['吾輩は猫である',  '本日は晴天なり']
embeddings = model.encode(sentences)

for i, embedding in enumerate(embeddings):
  print("[%d] : %s" % (i, embedding.shape, ))

Some weights of the model checkpoint at ../../corpus/cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[0] : (768,)
[1] : (768,)


In [ ]:
BATCH_SIZE = 32
NUM_EPOCHS = 3
EVAL_STEPS = 1000
WARMUP_STEPS = int(len(X_train) // BATCH_SIZE * 0.1) 
OUTPUT_PATH = "../../corpus/sbert_ignore"

In [ ]:
train_data = SentencesDataset(X_train, model=model)
train_dataloader = DataLoader(train_data, shuffle=True,  batch_size=BATCH_SIZE)
train_loss = losses.SoftmaxLoss(model=model, sentence_embedding_dimension=model.get_sentence_embedding_dimension(), num_labels=3)

In [ ]:
model.fit(train_objectives=[(train_dataloader, train_loss)],
         epochs=NUM_EPOCHS,
         evaluation_steps=EVAL_STEPS,
         warmup_steps=WARMUP_STEPS,
         output_path=OUTPUT_PATH
         )

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/26 [00:00<?, ?it/s]

RuntimeError: CUDA error: out of memory